# Working with the new Yr weather API

In [1]:
# some useful imports
import datetime, pytz
import requests
import json
import xmltodict
import xml.etree.ElementTree as ET
import emoji

In [2]:
lat = 53.86893
lon = 10.68729
alt = 162



In [3]:
class API_LocationForecast(object):

    """Class to use the API of api.met.no"""

    base_url = 'https://api.met.no/weatherapi/locationforecast/2.0/classic?'
    forecast_link = 'locationforecast'

    def __init__(self, lat, lon, alt):
        """
        :param double lat: latitude coordinate
        :param double lon: longitude coordinate
        """
        self.coordinates = dict(lat=lat, lon=lon, alt=alt)
        self.url = self.get_url()

    def get_url(self):
        """Return the url of API service"""
        return '{base_url}{location_name}'.format(
            base_url=self.base_url,
            location_name=self.location_name,
        )

In [4]:
# Create locationforecast url for API request

base_url = 'https://api.met.no/weatherapi/locationforecast/2.0/complete?'
api_url = '{base_url}lat={lat}&lon={lon}&altitude={alt}'.format(
    base_url=base_url,
    lat=lat,
    lon=lon,
    alt=alt
)

print(api_url)

https://api.met.no/weatherapi/locationforecast/2.0/complete?lat=53.86893&lon=10.68729&altitude=162


In [5]:
# Send request to API

sitename = 'https://jkl453.github.io/garden-123/'
headers = {'User-Agent': sitename}

response = requests.get(api_url, headers=headers)


print(response.status_code)
                                                                                                                                                                                                                                                             


200


In [6]:
#print(response.text)

response_json = json.loads(response.text)
print(json.dumps(response_json, indent=4))


{
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [
            10.6873,
            53.8689,
            162
        ]
    },
    "properties": {
        "meta": {
            "updated_at": "2022-06-08T16:35:26Z",
            "units": {
                "air_pressure_at_sea_level": "hPa",
                "air_temperature": "celsius",
                "air_temperature_max": "celsius",
                "air_temperature_min": "celsius",
                "air_temperature_percentile_10": "celsius",
                "air_temperature_percentile_90": "celsius",
                "cloud_area_fraction": "%",
                "cloud_area_fraction_high": "%",
                "cloud_area_fraction_low": "%",
                "cloud_area_fraction_medium": "%",
                "dew_point_temperature": "celsius",
                "fog_area_fraction": "%",
                "precipitation_amount": "mm",
                "precipitation_amount_max": "mm",
                "prec

**How to extract Data from json response**

In [7]:
json_snippet = response_json['properties']['timeseries']
print(json.dumps(json_snippet, indent=4))

[
    {
        "time": "2022-06-08T17:00:00Z",
        "data": {
            "instant": {
                "details": {
                    "air_pressure_at_sea_level": 1005.5,
                    "air_temperature": 21.9,
                    "air_temperature_percentile_10": 21.1,
                    "air_temperature_percentile_90": 22.6,
                    "cloud_area_fraction": 72.7,
                    "cloud_area_fraction_high": 72.1,
                    "cloud_area_fraction_low": 1.6,
                    "cloud_area_fraction_medium": 1.2,
                    "dew_point_temperature": 11.2,
                    "fog_area_fraction": 0.0,
                    "relative_humidity": 50.5,
                    "ultraviolet_index_clear_sky": 0.6,
                    "wind_from_direction": 133.7,
                    "wind_speed": 2.6,
                    "wind_speed_of_gust": 6.2,
                    "wind_speed_percentile_10": 1.9,
                    "wind_speed_percentile_90": 3.4
         

In [8]:
date_today = datetime.datetime.now(pytz.timezone("Europe/Berlin"))

print(date_today)

print(date_today.strftime("%c"))

date_today_day = date_today.strftime("%d")
date_today_month = date_today.strftime("%m")
date_today_time = date_today.strftime("%H:%M")

print("")
print(date_today_day)
print(date_today_month)
print(date_today_time)
print("")

todays_date = datetime.datetime.now(pytz.timezone("Europe/Berlin"))
tomorrows_date = todays_date + datetime.timedelta(days=1)

json_snippet = response_json['properties']['timeseries'][0]['time']
print(type(json_snippet))

print(datetime.datetime.strptime(json_snippet, '%Y-%m-%dT%H:%M:%SZ'))
print(json.dumps(json_snippet, indent=4))


# get weather datat from time
def get_weather_from_time(datetime_object):
    """
    Get weather data for a certain period of time
    :param string time: time of weather forecast
    :return: weather data
    """
    time_string = datetime.datetime.strftime(datetime_object, '%Y-%m-%dT%H:00:00Z')
    print(time_string)

    for item in response_json['properties']['timeseries']:
        if item['time'] == time_string:
            return json.dumps(item['data'], indent=4)

print(get_weather_from_time(tomorrows_date))

2022-06-08 19:00:06.546867+02:00
Wed Jun  8 19:00:06 2022

08
06
19:00

<class 'str'>
2022-06-08 17:00:00
"2022-06-08T17:00:00Z"
2022-06-09T19:00:00Z
{
    "instant": {
        "details": {
            "air_pressure_at_sea_level": 1015.1,
            "air_temperature": 15.8,
            "air_temperature_percentile_10": 14.4,
            "air_temperature_percentile_90": 17.8,
            "cloud_area_fraction": 69.8,
            "cloud_area_fraction_high": 0.0,
            "cloud_area_fraction_low": 69.8,
            "cloud_area_fraction_medium": 0.0,
            "dew_point_temperature": 11.0,
            "fog_area_fraction": 0.0,
            "relative_humidity": 73.0,
            "ultraviolet_index_clear_sky": 0.1,
            "wind_from_direction": 288.8,
            "wind_speed": 4.2,
            "wind_speed_of_gust": 8.4,
            "wind_speed_percentile_10": 3.6,
            "wind_speed_percentile_90": 4.6
        }
    },
    "next_12_hours": {
        "summary": {
            "s

# Funktionalität für Wettervorhersage

- [] Wetterbericht heute
- [] Wetterbericht morgen
- [] Wetterbericht für den nächsten persönlichen Gießtag

In der user-Datenbank kann dann immer auch der nächste Gießtag gespeichert werden. Dieser kann dann ausgelesen werden um den entsprechenden Wetterbericht zu generieren.

# Aufbau der Wettervorhersage

**individuelle Anfrage**
- aktueller Zeitpunkt + next 6 hours

**morgentliche Benachrichtigung**
- 06:00 Uhr bis 12:00 Uhr
- 12:00 Uhr bis 18:00 Uhr
- 18:00 Uhr bis 00:00 Uhr

In [9]:
"""individual forecast request"""

# get current date and time
date_today = datetime.datetime.now(pytz.timezone("Europe/Berlin"))
time_now = date_today.strftime("%H:%M")

print(time_now)
print(type(time_now))
print(date_today)

json_snippet = response_json['properties']['timeseries'][0]['data']['next_6_hours']
print(json.dumps(json_snippet, indent=4))

yr_symbol_code = json_snippet['summary']['symbol_code']
yr_precipitation_amount = json_snippet['details']['precipitation_amount']
yr_precipitation_probability = json_snippet['details']['probability_of_precipitation']


19:00
<class 'str'>
2022-06-08 19:00:06.569201+02:00
{
    "summary": {
        "symbol_code": "rain"
    },
    "details": {
        "air_temperature_max": 21.3,
        "air_temperature_min": 16.1,
        "precipitation_amount": 1.1,
        "precipitation_amount_max": 3.8,
        "precipitation_amount_min": 0.0,
        "probability_of_precipitation": 37.7
    }
}


In [10]:
# uni code weather symbols
# https://unicode.org/emoji/charts/full-emoji-list.html
weather_symbols_uni = {
    'sun': '\U00002600',
    'cloud': '\U00002601',
    'sun behind cloud': '\U000026C5',
    'cloud with lightning and rain': '\U000026C8',
    'sun behind small cloud': '\U0001F324',
    'sun behind large cloud': '\U0000F325',
    'sun behind rain cloud ': '\U0001F326',
    'cloud with rain': '\U0001F327',
    'cloud with snow': '\U0001F328',
    'cloud with lightning': '\U0001F329',
    'tornado': '\U0001F32A',
    'fog': '\U0001F32B',
    'wind face': '\U0001F32C'
}

# yr weather symbols numbers
# https://github.com/nrkno/yr-weather-symbols
weather_symbols_yr_numbers = {
    'sun':  ['01'],
    'cloud': ['04'],
    'sun behind cloud': ['03'],
    'cloud with lightning and rain': ['06', '24', '25', '26', '20', '27'],
    'sun behind small cloud': ['02'],
    'sun behind large cloud': [],
    'sun behind rain cloud ': ['40', '05', '41', '42', '07', '43'],
    'cloud with rain': ['46', '09', '10'],
    'cloud with snow': ['44', '08', '45', '28', '21', '29', '47', '12', '48', '49', '13', '50'],
    'cloud with lightning': ['30', '22', '11', '31', '23', '32', '33', '14', '34'],
    'fog': ['15']
}

# yr weather symbols names
# https://github.com/nrkno/yr-weather-symbols
weather_symbols_yr_names = {
    'sun':  ['clearsky_day',
             'clearsky_night'],
    'cloud': ['cloudy'],
    'sun behind cloud': ['partlycloudy_day',
                         'partlycloudy_night'],
    'cloud with lightning and rain': ['lightrainshowersandthunder_day',
                                      'lightrainshowersandthunder_night',
                                      'rainshowersandthunder_day',
                                      'rainshowersandthunder_night',
                                      'heavyrainshowersandthunder_day',
                                      'heavyrainshowersandthunder_night'],
    'sun behind small cloud': ['fair_day',
                               'fair_night'],
    'sun behind large cloud': [],
    'sun behind rain cloud ': ['lightrainshowers_day',
                               'lightrainshowers_night',
                               'rainshowers_day',
                               'rainshowers_night',
                               'heavyrainshowers_day',
                               'heavyrainshowers_night'],
    'cloud with rain': ['rain',
                        'heavyrain'],
    'cloud with snow': ['lightsnowshowers',
                        'snowshowers',
                        'heavysnowshowers',
                        'lightsnowshowersandthunder',
                        'snowshowersandthunder',
                        'heavysnowshowersandthunder',
                        'lightsleet',
                        'sleet',
                        'heavysleet',
                        'lightsnow',
                        'snow',
                        'heavysnow'],
    'cloud with lightning': ['lightrainandthunder',
                             'rainandthunder',
                             'heavyrainandthunder',
                             'lightsleetandthunder',
                             'sleetandthunder',
                             'heavysleetandthunder',
                             'lightsnowandthunder',
                             'snowandthunder',
                             'heavysnowandthunder'],
    'fog': ['fog']
}


In [11]:
print('Precipitation amount: ' + str(yr_precipitation_amount))
print('Precipitation probability: ' + str(yr_precipitation_probability) + ' %')

# get unicode of weather symbol
def get_uni_code(yr_symbol_code):
    search_name = yr_symbol_code
    for uni_name, yr_name_list in weather_symbols_yr_names.items():
        for yr_name in range(len(yr_name_list)):
            if yr_name_list[yr_name] == search_name:
                uni_code = weather_symbols_uni[uni_name]
                return uni_code

print(get_uni_code(yr_symbol_code))



Precipitation amount: 1.1
Precipitation probability: 37.7 %
🌧


In [12]:
# get current time
def get_time():
    time_now = datetime.datetime.now(pytz.timezone("Europe/Berlin"))
    time_now = time_now.strftime("%H:%M")
    return time_now

# get weather data from yr API
def get_weather_data():
        """
        Returns the weather data as a dictionary.
        """

        # send request to YR API
        response = requests.get(api_url, headers=headers)

        # convert response to JSON
        response_json = json.loads(response.text)
        weather_data_json = json.dumps(response_json, indent=4)

        return weather_data_json

# get weather forecast for next hour
def get_forecast(time, weather_data_json):
    """
    Returns the weather forecast for the next hour.
    """

    # get time
    time_now = time

    # get weather data
    weather_data = weather_data_json['properties']['timeseries']
    time_forecast = weather_data_json['properties']['timeseries'][0]['time']

    # get forecast for next hour
    for i in range(len(weather_data)):
        if weather_data[i]['validTime'] == time_now:
            forecast = weather_data[i]['data']['next_6_hours']
            break

    return forecast
    

print(get_time())

19:00


In [13]:
# get datetime object for tomorrow and day after tommorrow
def get_next_dates():
    date_tomorrow = datetime.datetime.now(pytz.timezone("Europe/Berlin")) + datetime.timedelta(days=1)
    date_after_tomorrow = datetime.datetime.now(pytz.timezone("Europe/Berlin")) + datetime.timedelta(days=2)
    return date_tomorrow, date_after_tomorrow

tomorrow_date, day_after_tomorrow_date = get_next_dates()

print(str(tomorrow_date) + ' and ' + str(day_after_tomorrow_date))

2022-06-09 19:00:06.685281+02:00 and 2022-06-10 19:00:06.685295+02:00
